<a href="https://colab.research.google.com/github/lohan20s/Predict_Spam_Message_LSTM/blob/main/fcc_sms_spam_classification_SL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
#load train and test data
train_dataset=pd.read_csv(train_file_path,sep="\t",header=None)
train_labels=train_dataset.pop(train_dataset.columns[0])
train_labels_int= train_labels.map({'ham': 1, 'spam': 0})

test_dataset=pd.read_csv(test_file_path,sep="\t",header=None)
test_labels=test_dataset.pop(test_dataset.columns[0])
test_labels_int= test_labels.map({'ham': 1, 'spam': 0})


In [ ]:
#encode words as integers by creating a dictionary from the full dataset
tokenizer = keras.preprocessing.text.Tokenizer(filters='') #ensure filters are off, so that phone numbers are not filtered out
combineddata=pd.concat([train_dataset,test_dataset])

tokenizer.fit_on_texts(combineddata.squeeze())
word_index = tokenizer.word_index  # Dictionary of words -> integer index
VOCAB_SIZE = len(word_index)
train_data = tokenizer.texts_to_sequences(train_dataset.squeeze())  # Convert sentences to sequences of integers
test_data = tokenizer.texts_to_sequences(test_dataset.squeeze())  # Convert sentences to sequences of integers


In [ ]:
#pad sequences so that all are the same length
max_train_length = max(len(seq) for seq in train_data)
max_test_length = max(len(seq) for seq in test_data)
MAXLEN=max(max_train_length,max_test_length)

Pad_sequence=keras.preprocessing.sequence.pad_sequences
train_data = Pad_sequence(train_data, MAXLEN)
test_data = Pad_sequence(test_data, MAXLEN)

In [ ]:
#create a model with LSTM for sequential data
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE+1, 64),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.summary()

In [ ]:
#compile and fit the model
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])
model.fit(train_data, train_labels_int, epochs=10,validation_data=(test_data, test_labels_int))
model.summary()

In [ ]:
#print accuracy of the model on the test dataset
results = model.evaluate(test_data, test_labels_int)
print(results)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  str=[pred_text]
  str_int=tokenizer.texts_to_sequences(str)
  str_int = keras.preprocessing.sequence.pad_sequences(str_int, MAXLEN)
  results=model.predict(str_int)
  prediction=[]
  prediction.append(results[0][0])
  if results[0][0]>0.5:
    prediction.append('ham')
  else:
    prediction.append('spam')
  return (prediction)

pred_text = "how are you doing today"
prediction = predict_message(pred_text)
print("The text message " + "is " + str(prediction[0]*100) +"% " + str(prediction[1]))

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()